In [1]:
%load_ext autotime

In [2]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

time: 932 µs


In [3]:
import magellan as mg
import numpy as np
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 2.27 s


In [4]:
# read in csv files
A= mg.read_csv('tableA.csv',key='id')
B= mg.read_csv('tableB.csv',key='id')

time: 180 ms


In [5]:
# get attribute list
attr_A= list(A.get_attr_names())
attr_B= list(B.get_attr_names())

time: 1.38 ms


In [6]:
# partition table into 2 based on whether attributes have NaN value
# return 2 tables in mg.MTable format
def partitionTable(table,attr):
    if type(table[attr][0]) != str:
        t1= table[~np.isnan(table[attr])]
        t2= table[np.isnan(table[attr])]
    else:
        t1= table[table[attr]!='']
        t2= table[table[attr]=='']        
    t1= mg.MTable(t1,key='id')
    t2= mg.MTable(t2,key='id')
    return t1, t2

time: 4.4 ms


In [7]:
# get partitioned table A based on zipcode
# A1 is the table with zipcode
# A2 is the table without zipcode
A1, A2= partitionTable(A,attr_A[5])

time: 37.1 ms


In [8]:
# Attribute equivalence blocking: block A1 and B using zipcode
ab = mg.AttrEquivalenceBlocker()
C1 = ab.block_tables(A1, B, attr_A[5], attr_B[5],
                    l_output_attrs=[attr_A[1],attr_A[2],attr_A[3],attr_A[4],attr_A[5]],
                    r_output_attrs=[attr_B[1],attr_B[2],attr_B[3],attr_B[4],attr_B[5]])

time: 1.97 s


In [9]:
C1.head()

,_id,ltable.id,rtable.id,ltable.name,ltable.streetAddress,ltable.city,ltable.state,ltable.zipCode,rtable.name,rtable.streetAddress,rtable.city,rtable.state,rtable.zipCode
0,0,1,1,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238
1,1,1,55,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Olea,171 Lafayette Ave,Brooklyn,NY,11238
2,2,1,62,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Alice’s Arbor,549 Classon Ave,Brooklyn,NY,11238
3,3,1,134,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Aita,132 Greene Ave,Brooklyn,NY,11238
4,4,1,141,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Cheryl’s Global Soul,236 Underhill Ave,Brooklyn,NY,11238


time: 126 ms


In [10]:
# Attribute equivalence blocking: block A2 and B using city
ab = mg.AttrEquivalenceBlocker()
C2 = ab.block_tables(A2, B, attr_A[3], attr_B[3],
                    l_output_attrs=[attr_A[1],attr_A[2],attr_A[3],attr_A[4], attr_A[5]],
                    r_output_attrs=[attr_B[1],attr_B[2],attr_B[3],attr_B[4], attr_B[5]])

time: 94 ms


In [11]:
# union tables C1 and C2 to C3
#----------- Pradap -------------------------
# The following is a hack -- this will work because A1 and A2 are strict subsets of A.
# set ltable to be A for C1 and C2
C1.set_property('ltable', A)
C2.set_property('ltable', A)


time: 1.58 ms


In [12]:
C1.columns

Index([u'_id', u'ltable.id', u'rtable.id', u'ltable.name',
       u'ltable.streetAddress', u'ltable.city', u'ltable.state',
       u'ltable.zipCode', u'rtable.name', u'rtable.streetAddress',
       u'rtable.city', u'rtable.state', u'rtable.zipCode'],
      dtype='object')

time: 2.19 ms


In [13]:
C2.columns

Index([u'_id', u'ltable.id', u'rtable.id', u'ltable.name',
       u'ltable.streetAddress', u'ltable.city', u'ltable.state',
       u'ltable.zipCode', u'rtable.name', u'rtable.streetAddress',
       u'rtable.city', u'rtable.state', u'rtable.zipCode'],
      dtype='object')

time: 1.94 ms


In [14]:
import pandas as pd

time: 880 µs


In [15]:
C1.drop('_id', inplace=True, axis=1)
C2.drop('_id', inplace=True, axis=1)

time: 98.2 ms


In [16]:
C3 = mg.MTable(pd.concat([C1, C2]), key='_id')

time: 2.28 s


In [17]:
C3

,_id,ltable.id,rtable.id,ltable.name,ltable.streetAddress,ltable.city,ltable.state,ltable.zipCode,rtable.name,rtable.streetAddress,rtable.city,rtable.state,rtable.zipCode
0,0,1,1,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238
1,1,1,55,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Olea,171 Lafayette Ave,Brooklyn,NY,11238
2,2,1,62,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Alice’s Arbor,549 Classon Ave,Brooklyn,NY,11238
3,3,1,134,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Aita,132 Greene Ave,Brooklyn,NY,11238
4,4,1,141,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Cheryl’s Global Soul,236 Underhill Ave,Brooklyn,NY,11238
5,5,1,214,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Mayfield,688 Franklin Ave,Brooklyn,NY,11238
6,6,1,262,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Silver Rice,638 Park Pl,Brooklyn,NY,11238
7,7,1,284,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Bunsmith,789 Franklin Ave,Brooklyn,NY,11238
8,8,1,287,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Marietta,285 Grand Ave,Brooklyn,NY,11238
9,9,1,305,Sunshine Co.,780 Washington Ave,Brooklyn,NY,11238,Citrico,681 Washington Ave,Brooklyn,NY,11238


time: 113 ms


In [18]:
C3.get_key()

'_id'

time: 2.28 ms


In [20]:
C3.set_property('ltable', A)
C3.set_property('rta')

time: 839 µs


In [21]:
ltable

time: 928 µs
